In [13]:
import torch
import torch.nn as nn

# Bi-GRU 모델 정의
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=4):
        super(BiGRU, self).__init__()
        self.bigru = nn.GRU(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
    
    def forward(self, x):
        output, hidden = self.bigru(x)
        return output, hidden

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=4):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True)
    
    def forward(self, x):
        output, hidden = self.gru(x)
        return output, hidden


# 차원 변화 확인
input_size = 100  # 입력 특성 차원
hidden_size = 256  # 은닉 상태 차원
seq_length = 20   # 시퀀스 길이
batch_size = 32   # 배치 크기

# 모델 생성
model0 = GRU(input_size, hidden_size)
model = BiGRU(input_size, hidden_size)

# 더미 입력 데이터 생성
input_data = torch.randn(batch_size, seq_length, input_size)

print(f"Batch_size: {batch_size}, \nSequence Length: {seq_length}")
print(f"Hidden: {hidden_size}")

# Bi-GRU를 통과시켜 representation 추출
with torch.no_grad():
    output0, hidden0 = model0(input_data)
    output, hidden = model(input_data)

fbhidden0 = hidden0[-1]  # GRU의 은닉 상태를 합칩니다.
fbhidden = torch.cat([hidden[-2], hidden[-1]], dim=-1)  # 양방향 GRU의 은닉 상태를 합칩니다.

print()
print("GRU----------------------")
print(f"입력 차원: {input_data.shape}")
print(f"출력 차원: {output0.shape}")
print(f"은닉 상태 차원: {hidden0.shape}")
print(f"마지막 은닉 상태 차원: {fbhidden0.shape}")

print()
print("Bi-GRU-------------------")
print(f"입력 차원: {input_data.shape}")
print(f"출력 차원: {output.shape}")
print(f"은닉 상태 차원: {hidden.shape}")
print(f"마지막 은닉 상태 차원: {fbhidden.shape}")

Batch_size: 32, 
Sequence Length: 20
Hidden: 256

GRU----------------------
입력 차원: torch.Size([32, 20, 100])
출력 차원: torch.Size([32, 20, 256])
은닉 상태 차원: torch.Size([4, 32, 256])
마지막 은닉 상태 차원: torch.Size([32, 256])

Bi-GRU-------------------
입력 차원: torch.Size([32, 20, 100])
출력 차원: torch.Size([32, 20, 512])
은닉 상태 차원: torch.Size([8, 32, 256])
마지막 은닉 상태 차원: torch.Size([32, 512])


In [5]:
import torch

x = torch.randn(10, 20)  # 예시로 랜덤 텐서 생성

import torch.nn as nn

sm = nn.Softmax(dim=1)

res = sm(x)

print(x.shape)
print(res.shape)

torch.Size([10, 20])
torch.Size([10, 20])
